In [1]:
!pip install datasets==3.6.0 --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 28.3 MB/s eta 0:00:00


In [ ]:
!git config --global user.email "aravindrajeshmenon2003@gmail.com"
!git config --global user.name "lonelyalpaca"

!git clone https://github.com/lonelyalpaca2003/Multi-language-NLP-pipeline.git
%cd Multi-language-NLP-pipeline

Cloning into 'Multi-language-NLP-pipeline'...
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 7 (delta 0), reused 4 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (7/7), 5.36 KiB | 5.36 MiB/s, done.
/content/Multi-language-NLP-pipeline


In [2]:
!pip install evaluate --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.7 MB/s eta 0:00:00


In [3]:
import datasets
import json

In [19]:
from datasets import load_dataset, concatenate_datasets, Dataset
import pandas as pd
import numpy as np
import os
import evaluate
from transformers import XLMRobertaTokenizer, XLMRobertaPreTrainedModel, DataCollatorWithPadding
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from transformers import AutoModelForSequenceClassification
from transformers import EarlyStoppingCallback
from huggingface_hub import notebook_login
notebook_login()

In [5]:
dfs = []
base_path = '/content/drive/MyDrive/multi-language-nlp'
for path in os.listdir(base_path):
  dfs.append(pd.read_csv(f'{base_path}/{path}'))

In [6]:
malayalam_df = pd.concat(dfs)
print(malayalam_df.isna().sum())
malayalam_df = malayalam_df[['content', 'clean_content', 'english', 'sentiment']]
malayalam_df = malayalam_df.dropna()
print(malayalam_df.shape)
malayalam = Dataset.from_pandas(malayalam_df)

datetimee         273
usertag             0
username            0
content             2
clean_content       3
english             3
sentiment           0
datetime         8967
dtype: int64
(9237, 4)


In [7]:
malayalam

Dataset({
    features: ['content', 'clean_content', 'english', 'sentiment', '__index_level_0__'],
    num_rows: 9237
})

In [ ]:
#english = load_dataset("sentiment140", trust_remote_code = True)

In [ ]:
#english

DatasetDict({
    train: Dataset({
        features: ['text', 'date', 'user', 'sentiment', 'query'],
        num_rows: 1600000
    })
    test: Dataset({
        features: ['text', 'date', 'user', 'sentiment', 'query'],
        num_rows: 498
    })
})

In [28]:
french = load_dataset("CATIE-AQ/french_book_reviews_fr_prompt_sentiment_analysis")

README.md: 0.00B [00:00, ?B/s]

train.parquet:   0%|          | 0.00/12.1M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/270424 [00:00<?, ? examples/s]

In [29]:
french

DatasetDict({
    train: Dataset({
        features: ['inputs', 'targets'],
        num_rows: 270424
    })
})

In [5]:
chinese = load_dataset("tyqiangz/multilingual-sentiments","chinese")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

multilingual-sentiments.py: 0.00B [00:00, ?B/s]

chinese/train/0000.parquet:   0%|          | 0.00/13.5M [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/330k [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/339k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3000 [00:00<?, ? examples/s]

In [ ]:
chinese

DatasetDict({
    train: Dataset({
        features: ['text', 'source', 'label'],
        num_rows: 120000
    })
    validation: Dataset({
        features: ['text', 'source', 'label'],
        num_rows: 3000
    })
    test: Dataset({
        features: ['text', 'source', 'label'],
        num_rows: 3000
    })
})

In [10]:
hindi = load_dataset("OdiaGenAI/sentiment_analysis_hindi", trust_remote_code= True, )

README.md:   0%|          | 0.00/640 [00:00<?, ?B/s]

sentiment_analysis_term_train.jsonl: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/2497 [00:00<?, ? examples/s]

In [11]:
malayalam2 = load_dataset("wlkla/Malayalam_first_ready_for_sentiment", trust_remote_code= True)

README.md:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

Malayalam_first_ready_for_sentiment.tsv: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/6739 [00:00<?, ? examples/s]

In [5]:
spanish = load_dataset('azherali/enlish_spanish_amazon-books-reviews', trust_remote_code=True)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/792 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/3.15M [00:00<?, ?B/s]

data/validation-00000-of-00001.parquet:   0%|          | 0.00/77.2k [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/88.1k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/17612 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/424 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/442 [00:00<?, ? examples/s]

In [6]:
spanish = spanish.filter(lambda x: x['language'] == 'es')

Filter:   0%|          | 0/17612 [00:00<?, ? examples/s]

Filter:   0%|          | 0/424 [00:00<?, ? examples/s]

Filter:   0%|          | 0/442 [00:00<?, ? examples/s]

In [7]:
spanish2 = load_dataset('NickyNicky/Finance_sentiment_and_topic_classification_Translation_English_to_Spanish_v1', trust_remote_code = True)

README.md:   0%|          | 0.00/698 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/8.86M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/43830 [00:00<?, ? examples/s]

In [48]:
spanish2

DatasetDict({
    train: Dataset({
        features: ['user_prompt_en', 'user_prompt_es', 'answer', 'system_prompt', 'task_type'],
        num_rows: 43830
    })
})

Now that we've loaded the datasets to build the multilanguage pipeline, the next step is to standardise all of the datasets to have the same columns and merge them into a single dataset. In addition to this, we also see that while languages such as Chinese and English have a lot of text to work on, languages such as Hindi, Malayalam and French have way fewer marked examples that are ready for sentiment analysis. We will address this in the modelling part.

In [8]:
spanish = spanish.map(lambda x: {'language': 'spanish'})
spanish2 = spanish2.map(lambda x : {'language' : 'spanish'})
#english = english.map(lambda x: {'language': 'english'})
#malayalam = malayalam.map(lambda x: {'language': 'malayalam'})
#malayalam2 = malayalam2.map(lambda x : {'language' : 'malayalam'})
#french = french.map(lambda x: {'language': 'french'})
#chinese = chinese.map(lambda x: {'language': 'chinese'})
#hindi = hindi.map(lambda x: {'language': 'hindi'})

Map:   0%|          | 0/7107 [00:00<?, ? examples/s]

Map:   0%|          | 0/193 [00:00<?, ? examples/s]

Map:   0%|          | 0/164 [00:00<?, ? examples/s]

Map:   0%|          | 0/43830 [00:00<?, ? examples/s]

In [9]:
def standardise_dataframe(dataset, data_column, label):
  dataset = dataset.rename_column(data_column, 'input').rename_column(label, 'labels')
  return dataset.select_columns(['input','labels', 'language'])


In [10]:
spanish = standardise_dataframe(spanish['train'],'review_body', 'stars')
spanish2 = standardise_dataframe(spanish2['train'], 'user_prompt_es', 'answer')
#english = standardise_dataframe(english['train'], 'text', 'sentiment')
#malayalam = standardise_dataframe(malayalam, 'clean_content', 'sentiment')
#malayalam2 = standardise_dataframe(malayalam2['train'], 'query', 'label')
#french = standardise_dataframe(french['train'], 'inputs', 'targets')
#chinese = standardise_dataframe(chinese['train'], 'text', 'label')
#hindi = standardise_dataframe(hindi['train'], 'text', 'label')

In [11]:
def standardise_labels(example):
  label_value = example['labels']

  mapped_label = None
  if isinstance(label_value, int):
    if label_value in [0,1,2]:
      mapped_label = label_value

    elif label_value <= 2:
      mapped_label = 0
    elif label_value == 3:
      mapped_label = 0.5
    else:
      mapped_label = 1

  elif isinstance(label_value, str):
    label_map = {"neg" : 0, 'Negative': 0,
                 "neu" : 0.5, 'unknown_state' : 0.5,
                 "pos" : 1, 'Positive' : 1}
    mapped_label = label_map.get(label_value, 0.5)

  else:
    mapped_label = 0.5

  return {'label': int(mapped_label)}

In [12]:
spanish = spanish.map(standardise_labels, remove_columns=['labels'])
spanish2 = spanish2.map(standardise_labels, remove_columns = ['labels'])
#english = english.map(standardise_labels, remove_columns=['labels'])
#malayalam = malayalam.map(standardise_labels, remove_columns=['labels'])
#malayalam2 = malayalam2.map(standardise_labels, remove_columns= ['labels'])
#french = french.map(standardise_labels, remove_columns=['labels'])
#chinese = chinese.map(standardise_labels, remove_columns=['labels'])
#hindi = hindi.map(standardise_labels, remove_columns=['labels'])

Map:   0%|          | 0/7107 [00:00<?, ? examples/s]

Map:   0%|          | 0/43830 [00:00<?, ? examples/s]

In [55]:
spanish

Dataset({
    features: ['input', 'language', 'label'],
    num_rows: 7107
})

In [13]:
spanish_df = concatenate_datasets([spanish, spanish2])

Now there are several ways to analyse this dataset. We will try to answer two main questions: does a model trained on higher resource languages (such as English or French) transfer well to other lower resource languages? Does a model trained on English learn enough about the script to help analyse languages with similar scripts (such as Spanish or some other European language). Secondly, we can also check whether specialised languages can outperform a more resource-heavy generalised model in tokenizing and sentiment analysis tasks. In particular, we ask if a model specialised for a single lower resource language (such as Malayalam) can be used instead of using a single large model such as XLM-RoBERTa.

In [ ]:
english_df = df.filter(lambda x: x['language'] == 'english')
english_subset = english_df.shuffle(seed = 42).select(range(1000000))

In [ ]:
splits = english_subset.train_test_split(test_size = 0.3, seed = 42)
train_english = splits['train']
test_english = splits['test']

val_split = test_english.train_test_split(test_size = 0.5, seed = 42)
val_english = val_split['train']
test_english = val_split['test']

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('xlm-roberta-base')

def tokenize(text):
  return tokenizer(text['input'], padding = True, max_length = 512, truncation = True)

train_inputs = train_english.map(tokenize, batched = True)
test_inputs = test_english.map(tokenize, batched = True)
val_inputs = val_english.map(tokenize, batched = True)

In [17]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained('xlm-roberta-base', num_labels = 3)

In [16]:
def compute_metrics(eval_pred):
    load_accuracy = evaluate.load("accuracy")
    load_f1 = evaluate.load("f1")

    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
    f1 = load_f1.compute(predictions=predictions, references=labels, average="weighted")["f1"]

    return {"accuracy": accuracy, "f1": f1}

In [ ]:
repo_name = "finetuning-sentiment-model"

training_args = TrainingArguments(
   output_dir=repo_name,
   learning_rate=2e-5,
   per_device_train_batch_size=32,
   per_device_eval_batch_size=16,
   num_train_epochs=2,
   save_strategy = "steps",
   eval_strategy = "steps",
   save_steps = 5000,
   eval_steps = 5000,
   weight_decay=0.01,
   push_to_hub=True,
   metric_for_best_model = 'accuracy',
   load_best_model_at_end = True
)

trainer = Trainer(
   model=model,
   args=training_args,
   train_dataset=train_inputs,
   eval_dataset=test_inputs,
   processing_class=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics,
   callbacks = [EarlyStoppingCallback(early_stopping_patience = 3)]
)

In [ ]:
trainer.train()

In [ ]:
trainer.push_to_hub()

In [ ]:
trainer.evaluate()

In [14]:
model = AutoModelForSequenceClassification.from_pretrained("lonelyalpaca/finetuning-sentiment-model")
tokenizer = AutoTokenizer.from_pretrained("lonelyalpaca/finetuning-sentiment-model")

config.json:   0%|          | 0.00/879 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

In [15]:
def tokenize(text):
  return tokenizer(text['input'], padding = True, max_length = 512, truncation = True)

In [24]:
french_subset = french.train_test_split(test_size = 0.3, seed = 42)
french_test = french_subset['test']
chinese_subset = chinese.train_test_split(test_size = 0.3, seed = 42)
chinese_test = chinese_subset['test']

In [27]:
malayalam_tokenized = malayalam.map(tokenize, batched = True)
french_tokenized = french.map(tokenize, batched = True)
chinese_tokenized = chinese.map(tokenize, batched = True)
hindi_tokenized = hindi.map(tokenize, batched = True)
spanish_tokenized = spanish.map(tokenize, batched = True)


Map:   0%|          | 0/9237 [00:00<?, ? examples/s]

Map:   0%|          | 0/270424 [00:00<?, ? examples/s]

Map:   0%|          | 0/120000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2497 [00:00<?, ? examples/s]

Map:   0%|          | 0/7107 [00:00<?, ? examples/s]

## Zero Shot Transfer

In [ ]:
zero_shot_f1 = {}

In [ ]:
trainer = Trainer(
    model=model,
    compute_metrics=compute_metrics,
)

In [ ]:
languages = ['french', 'chinese', 'hindi', 'spanish']
datasets = [french_tokenized, chinese_tokenized, hindi_tokenized, spanish_tokenized]

for lang, dataset in zip(languages, datasets):
  result = trainer.evaluate(dataset)
  zero_shot_f1[lang] = result['eval_f1']

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: aravindrajeshmenon2003 (aravindrajeshmenon2003-london-school-of-economics-and-po) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [ ]:
zero_shot_f1

{'french': 0.8786807914960252,
 'chinese': 0.12688155623303393,
 'hindi': 0.6116529600856309,
 'spanish': 0.5518354312994006}

In [ ]:
with open("zero_shot_results.json", "w") as f:
  json.dump(zero_shot_f1, f, indent = 2)

## Fine-Tuning on Individual Languages

In [25]:
french_subset = french.shuffle(seed = 42).select(range(50000))

In [20]:
def split_dataset(dataset):
  split = dataset.train_test_split(test_size = 0.3, seed = 42)
  train_data = split['train']
  test_data = split['test']
  val_split = test_data.train_test_split(test_size = 0.5, seed = 42)
  val_data = val_split['train']
  test_data = val_split['test']
  train_data = train_data.map(tokenize, batched = True)
  test_data = test_data.map(tokenize, batched = True)
  val_data = val_data.map(tokenize, batched = True)
  return train_data, val_data, test_data

In [27]:
french_train, french_val, french_test = split_dataset(french_subset)

Map:   0%|          | 0/35000 [00:00<?, ? examples/s]

Map:   0%|          | 0/7500 [00:00<?, ? examples/s]

Map:   0%|          | 0/7500 [00:00<?, ? examples/s]

In [28]:
repo_name = "finetuning-sentiment-model-2"

french_args = TrainingArguments(
   output_dir=repo_name,
   learning_rate=2e-5,
   per_device_train_batch_size=32,
   per_device_eval_batch_size=16,
   num_train_epochs=2,
   save_strategy = "steps",
   eval_strategy = "steps",
   save_steps = 1000,
   eval_steps = 1000,
   weight_decay=0.01,
   push_to_hub=True,
   metric_for_best_model = 'accuracy',
   load_best_model_at_end = True
)

In [29]:
trainer = Trainer(model = model,
                  args = french_args,
                  train_dataset = french_train,
                  eval_dataset = french_val,
                  processing_class = tokenizer,
                  data_collator=data_collator,
                  compute_metrics=compute_metrics,
                  callbacks = [EarlyStoppingCallback(early_stopping_patience = 2)]
                  )

In [30]:
trainer.train()

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: aravindrajeshmenon2003 (aravindrajeshmenon2003-london-school-of-economics-and-po) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss,Validation Loss,Accuracy,F1
1000,0.124200,0.090565,0.976400,0.975005
2000,0.050300,0.057357,0.986933,0.986694


TrainOutput(global_step=2188, training_loss=0.09927195997081248, metrics={'train_runtime': 6584.3957, 'train_samples_per_second': 10.631, 'train_steps_per_second': 0.332, 'total_flos': 1.841793924096e+16, 'train_loss': 0.09927195997081248, 'epoch': 2.0})

In [31]:
trainer.evaluate()

{'eval_loss': 0.05735746771097183,
 'eval_accuracy': 0.9869333333333333,
 'eval_f1': 0.9866938809495007,
 'eval_runtime': 198.5852,
 'eval_samples_per_second': 37.767,
 'eval_steps_per_second': 2.362,
 'epoch': 2.0}

In [33]:
trainer.push_to_hub()

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...model-2/training_args.bin: 100%|##########| 5.84kB / 5.84kB            

  ...675117.2c48361b765d.682.0: 100%|##########| 7.25kB / 7.25kB            

  ...2/sentencepiece.bpe.model: 100%|##########| 5.07MB / 5.07MB            

  ...model-2/model.safetensors:   4%|3         | 41.9MB / 1.11GB            

  ...nt-model-2/tokenizer.json: 100%|##########| 17.1MB / 17.1MB            

  ...681916.2c48361b765d.682.1: 100%|##########|   457B /   457B            

CommitInfo(commit_url='https://huggingface.co/lonelyalpaca/finetuning-sentiment-model-2/commit/80383a2569ef51ed7be5b47642c36860fae72aa9', commit_message='End of training', commit_description='', oid='80383a2569ef51ed7be5b47642c36860fae72aa9', pr_url=None, repo_url=RepoUrl('https://huggingface.co/lonelyalpaca/finetuning-sentiment-model-2', endpoint='https://huggingface.co', repo_type='model', repo_id='lonelyalpaca/finetuning-sentiment-model-2'), pr_revision=None, pr_num=None)

### Chinese

In [14]:
chinese_subset = chinese.shuffle(seed = 42).select(range(50000))

In [17]:
chn_train, chn_val, chn_test = split_dataset(chinese)

Map:   0%|          | 0/84000 [00:00<?, ? examples/s]

Map:   0%|          | 0/18000 [00:00<?, ? examples/s]

Map:   0%|          | 0/18000 [00:00<?, ? examples/s]

In [18]:
repo_name = "finetuning-sentiment-model-chn"

chn_args = TrainingArguments(
   output_dir=repo_name,
   learning_rate=2e-5,
   per_device_train_batch_size=32,
   per_device_eval_batch_size=16,
   num_train_epochs=2,
   save_strategy = "steps",
   eval_strategy = "steps",
   save_steps = 1000,
   eval_steps = 1000,
   weight_decay=0.01,
   push_to_hub=True,
   metric_for_best_model = 'accuracy',
   load_best_model_at_end = True
)

In [24]:
trainer = Trainer(model = model,
                  args = chn_args,
                  train_dataset =  chn_train,
                  eval_dataset = chn_val,
                  processing_class = tokenizer,
                  data_collator=data_collator,
                  compute_metrics=compute_metrics,
                  callbacks = [EarlyStoppingCallback(early_stopping_patience = 2)]
                  )

In [25]:
trainer.train()

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: aravindrajeshmenon2003 (aravindrajeshmenon2003-london-school-of-economics-and-po) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss,Validation Loss,Accuracy,F1
1000,0.577200,0.548225,0.777278,0.778169
2000,0.547400,0.528374,0.784556,0.783771
3000,0.498800,0.545049,0.786444,0.782893
4000,0.475100,0.507584,0.796889,0.795611
5000,0.468600,0.503228,0.798111,0.795212


TrainOutput(global_step=5250, training_loss=0.5577055446079799, metrics={'train_runtime': 16702.1514, 'train_samples_per_second': 10.059, 'train_steps_per_second': 0.314, 'total_flos': 4.4203054178304e+16, 'train_loss': 0.5577055446079799, 'epoch': 2.0})

In [26]:
finetuned_f1 = {}
result = trainer.evaluate()
finetuned_f1['chinese'] = result['eval_f1']

In [ ]:
trainer.push_to_hub()

## Spanish

In [21]:
spanish_train, spanish_val, spanish_test = split_dataset(spanish_df)

Map:   0%|          | 0/35655 [00:00<?, ? examples/s]

Map:   0%|          | 0/7641 [00:00<?, ? examples/s]

Map:   0%|          | 0/7641 [00:00<?, ? examples/s]

In [22]:
repo_name = "finetuning-sentiment-model-spanish"

spn_args = TrainingArguments(
   output_dir=repo_name,
   learning_rate=1e-5,
   per_device_train_batch_size=16, # Reduced from 32
   per_device_eval_batch_size=8, # Reduced from 16
   num_train_epochs=2,
   save_strategy = "steps",
   eval_strategy = "steps",
   save_steps = 1000,
   eval_steps = 1000,
   weight_decay=0.01,
   push_to_hub=True,
   metric_for_best_model = 'accuracy',
   load_best_model_at_end = True
)

In [23]:
trainer = Trainer(model = model,
                  args = spn_args,
                  train_dataset =  spanish_train,
                  eval_dataset = spanish_val,
                  processing_class = tokenizer,
                  data_collator=data_collator,
                  compute_metrics=compute_metrics,
                  callbacks = [EarlyStoppingCallback(early_stopping_patience = 2)]
                  )

In [24]:
trainer.train()

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: aravindrajeshmenon2003 (aravindrajeshmenon2003-london-school-of-economics-and-po) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss,Validation Loss,Accuracy,F1
1000,0.163200,0.162391,0.941369,0.929480
2000,0.136100,0.125357,0.945295,0.935644
3000,0.116000,0.145134,0.946342,0.943798
4000,0.117500,0.133743,0.948174,0.948345


TrainOutput(global_step=4458, training_loss=0.1463639707295763, metrics={'train_runtime': 7222.4771, 'train_samples_per_second': 9.873, 'train_steps_per_second': 0.617, 'total_flos': 1.876259315130077e+16, 'train_loss': 0.1463639707295763, 'epoch': 2.0})

In [25]:
finetuned_f1 = {}
result = trainer.evaluate()
print(result)
finetuned_f1['spanish'] = result['eval_f1']

{'eval_loss': 0.13374310731887817, 'eval_accuracy': 0.9481743227326266, 'eval_f1': 0.9483448866135418, 'eval_runtime': 185.1175, 'eval_samples_per_second': 41.276, 'eval_steps_per_second': 5.164, 'epoch': 2.0}


In [26]:
trainer.push_to_hub()

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...spanish/training_args.bin: 100%|##########| 5.91kB / 5.91kB            

  ...718743.840c5532bbda.372.0: 100%|##########| 8.84kB / 8.84kB            

  ...h/sentencepiece.bpe.model: 100%|##########| 5.07MB / 5.07MB            

  ...spanish/model.safetensors:   4%|3         | 41.9MB / 1.11GB            

  ...el-spanish/tokenizer.json: 100%|##########| 17.1MB / 17.1MB            

  ...726176.840c5532bbda.372.1: 100%|##########|   457B /   457B            

CommitInfo(commit_url='https://huggingface.co/lonelyalpaca/finetuning-sentiment-model-spanish/commit/7f559eae69d161ce34ccea2253bcbd37ad0ab1b6', commit_message='End of training', commit_description='', oid='7f559eae69d161ce34ccea2253bcbd37ad0ab1b6', pr_url=None, repo_url=RepoUrl('https://huggingface.co/lonelyalpaca/finetuning-sentiment-model-spanish', endpoint='https://huggingface.co', repo_type='model', repo_id='lonelyalpaca/finetuning-sentiment-model-spanish'), pr_revision=None, pr_num=None)